# Решающие деревья (Decision Trees)

## Введение

Решающее дерево — это алгоритм машинного обучения, который принимает решения путем разбиения пространства признаков на регионы. Это один из самых интерпретируемых алгоритмов ML.

### Применение в биологии:
- Диагностика заболеваний по симптомам и анализам
- Классификация типов опухолей по экспрессии генов
- Предсказание эффективности лекарств
- Анализ факторов риска заболеваний

### Ключевые концепции:
- **Высокий bias, низкий variance**: Мелкое дерево (underfitting)
- **Низкий bias, высокий variance**: Глубокое дерево (overfitting)
- **Bias-Variance Tradeoff**: Балансировка через глубину дерева

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score
import pandas as pd

np.random.seed(42)

## 1. Как работает решающее дерево

### Принцип работы:

1. **Выбор лучшего разбиения**: На каждом узле выбирается признак и порог, которые лучше всего разделяют данные
2. **Критерий разбиения**:
   - Для регрессии: минимизация MSE (Mean Squared Error)
   - Для классификации: минимизация Gini impurity или энтропии
3. **Рекурсивное разбиение**: Процесс повторяется для каждого поддерева
4. **Остановка**: По критериям (максимальная глубина, минимум объектов в листе и т.д.)

### Критерий разбиения для регрессии:

В узле $m$ ищем признак $j$ и порог $t$, которые минимизируют:

$$\text{MSE}_{left}(j, t) + \text{MSE}_{right}(j, t)$$

где:
$$\text{MSE} = \frac{1}{n}\sum_{i=1}^{n}(y_i - \bar{y})^2$$

## 2. Пример: Регрессия (предсказание концентрации препарата)

In [ ]:
# Генерируем данные: нелинейная зависимость концентрации от времени
def true_function(x):
    """Истинная нелинейная функция"""
    return np.sin(x) * x + 0.5 * x

# Тренировочные данные
n_samples = 100
X_train = np.sort(np.random.uniform(0, 10, n_samples))
y_train = true_function(X_train) + np.random.normal(0, 0.5, n_samples)

# Тестовые данные
X_test = np.linspace(0, 10, 200)
y_test_true = true_function(X_test)

X_train_2d = X_train.reshape(-1, 1)
X_test_2d = X_test.reshape(-1, 1)

print(f"Размер тренировочной выборки: {n_samples}")
print(f"Диапазон времени: [0, 10] часов")

### Визуализация влияния глубины дерева

In [ ]:
# Создаем деревья разной глубины
depths = [1, 2, 5, 20]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, depth in enumerate(depths):
    # Обучаем дерево
    tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
    tree.fit(X_train_2d, y_train)
    
    # Предсказания
    y_pred = tree.predict(X_test_2d)
    
    # Ошибка на тренировочной выборке
    train_mse = mean_squared_error(y_train, tree.predict(X_train_2d))
    
    # График
    axes[idx].scatter(X_train, y_train, alpha=0.4, s=20, label='Тренировочные данные')
    axes[idx].plot(X_test, y_test_true, 'g--', lw=2, label='Истинная функция')
    axes[idx].plot(X_test, y_pred, 'r-', lw=2, label=f'Предсказание (depth={depth})')
    axes[idx].set_xlabel('Время (часы)')
    axes[idx].set_ylabel('Концентрация')
    axes[idx].set_title(f'Глубина = {depth}, Train MSE = {train_mse:.3f}', fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    
    # Добавляем комментарий
    if depth == 1:
        axes[idx].text(0.5, 0.95, 'Высокий BIAS\n(underfitting)', 
                      transform=axes[idx].transAxes, fontsize=10, 
                      verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    elif depth == 20:
        axes[idx].text(0.5, 0.95, 'Высокий VARIANCE\n(overfitting)', 
                      transform=axes[idx].transAxes, fontsize=10, 
                      verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))

plt.tight_layout()
plt.show()

## 3. Bias-Variance Tradeoff

### Теория:

Ожидаемая ошибка модели разлагается на три компоненты:

$$\text{Expected Error} = \text{Bias}^2 + \text{Variance} + \text{Irreducible Error}$$

- **Bias (смещение)**: Ошибка из-за упрощающих предположений модели
  - Высокий bias → underfitting
  - Модель слишком простая
  
- **Variance (разброс)**: Ошибка из-за чувствительности к флуктуациям в данных
  - Высокий variance → overfitting
  - Модель слишком сложная
  
- **Irreducible Error**: Шум в данных (не зависит от модели)

### Для решающих деревьев:

| Параметр | Bias | Variance |
|----------|------|----------|
| Маленькая глубина | ↑ Высокий | ↓ Низкий |
| Большая глубина | ↓ Низкий | ↑ Высокий |
| Много объектов в листе | ↑ Высокий | ↓ Низкий |
| Мало объектов в листе | ↓ Низкий | ↑ Высокий |

### Демонстрация Bias-Variance Tradeoff

In [ ]:
# Функция для вычисления bias и variance
def compute_bias_variance(max_depth, n_iterations=100):
    """
    Вычисляет bias и variance для дерева заданной глубины
    путем обучения на разных подвыборках данных
    """
    predictions = np.zeros((n_iterations, len(X_test)))
    
    for i in range(n_iterations):
        # Генерируем новую выборку
        X_sample = np.sort(np.random.uniform(0, 10, n_samples))
        y_sample = true_function(X_sample) + np.random.normal(0, 0.5, n_samples)
        
        # Обучаем дерево
        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=i)
        tree.fit(X_sample.reshape(-1, 1), y_sample)
        
        # Предсказания
        predictions[i] = tree.predict(X_test_2d)
    
    # Среднее предсказание по всем моделям
    mean_prediction = np.mean(predictions, axis=0)
    
    # Bias^2: (среднее предсказание - истинное значение)^2
    bias_squared = np.mean((mean_prediction - y_test_true)**2)
    
    # Variance: разброс предсказаний разных моделей
    variance = np.mean(np.var(predictions, axis=0))
    
    return bias_squared, variance

# Вычисляем bias и variance для разных глубин
depths_range = range(1, 16)
biases = []
variances = []

print("Вычисление bias и variance для разных глубин...")
for depth in depths_range:
    bias_sq, var = compute_bias_variance(depth, n_iterations=50)
    biases.append(bias_sq)
    variances.append(var)
    print(f"Глубина {depth:2d}: Bias² = {bias_sq:.3f}, Variance = {var:.3f}")

print("\nГотово!")

### График Bias-Variance Tradeoff

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(depths_range, biases, 'b-o', lw=2, markersize=6, label='Bias²')
ax.plot(depths_range, variances, 'r-s', lw=2, markersize=6, label='Variance')
total_error = np.array(biases) + np.array(variances)
ax.plot(depths_range, total_error, 'g-^', lw=2.5, markersize=6, label='Total Error (Bias² + Variance)')

# Находим оптимальную глубину
optimal_depth = list(depths_range)[np.argmin(total_error)]
ax.axvline(optimal_depth, color='purple', linestyle='--', lw=2, alpha=0.7, 
           label=f'Оптимальная глубина ≈ {optimal_depth}')

ax.set_xlabel('Максимальная глубина дерева', fontsize=12)
ax.set_ylabel('Ошибка', fontsize=12)
ax.set_title('Bias-Variance Tradeoff для решающих деревьев', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Добавляем аннотации
ax.annotate('Underfitting\n(Высокий Bias)', xy=(2, biases[1]), xytext=(3, biases[1]+1),
            arrowprops=dict(arrowstyle='->', color='blue', lw=1.5),
            fontsize=10, color='blue', fontweight='bold')

ax.annotate('Overfitting\n(Высокий Variance)', xy=(14, variances[-2]), xytext=(11, variances[-2]+1.5),
            arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
            fontsize=10, color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n✓ Оптимальная глубина дерева: {optimal_depth}")
print(f"  Минимальная ошибка: {min(total_error):.3f}")

## 4. Пример классификации: Диагностика заболевания

Классифицируем пациентов на больных/здоровых по двум биомаркерам.

In [ ]:
from sklearn.datasets import make_moons

# Генерируем данные (нелинейно разделимые классы)
X_class, y_class = make_moons(n_samples=300, noise=0.25, random_state=42)

# Разделяем на train/test
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_class, y_class, test_size=0.3, random_state=42
)

print(f"Тренировочная выборка: {len(X_train_c)} пациентов")
print(f"Тестовая выборка: {len(X_test_c)} пациентов")
print(f"Классы: 0 (здоров), 1 (болен)")

### Визуализация границ решений для разных глубин

In [ ]:
def plot_decision_boundary(model, X, y, ax, title):
    """Визуализирует границу решения классификатора"""
    h = 0.02  # шаг сетки
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu', edgecolors='black', s=40)
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel('Биомаркер 1')
    ax.set_ylabel('Биомаркер 2')

# Создаем классификаторы разной сложности
depths_class = [2, 4, 8, None]  # None = без ограничения глубины

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for idx, depth in enumerate(depths_class):
    # Обучаем дерево
    tree_clf = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree_clf.fit(X_train_c, y_train_c)
    
    # Точность
    train_acc = accuracy_score(y_train_c, tree_clf.predict(X_train_c))
    test_acc = accuracy_score(y_test_c, tree_clf.predict(X_test_c))
    
    # Визуализация
    depth_str = str(depth) if depth is not None else '∞'
    title = f'Глубина = {depth_str}\nTrain Acc = {train_acc:.2%}, Test Acc = {test_acc:.2%}'
    plot_decision_boundary(tree_clf, X_train_c, y_train_c, axes[idx], title)
    
    # Добавляем комментарий
    if depth == 2:
        axes[idx].text(0.5, 0.02, 'Высокий BIAS', transform=axes[idx].transAxes, 
                      fontsize=10, ha='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))
    elif depth is None:
        axes[idx].text(0.5, 0.02, 'Высокий VARIANCE', transform=axes[idx].transAxes, 
                      fontsize=10, ha='center', bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))

plt.tight_layout()
plt.show()

## 5. Кривые обучения (Learning Curves)

Кривые обучения показывают, как ошибка на тренировочной и валидационной выборках зависит от размера данных.

In [ ]:
def plot_learning_curves(max_depth):
    """Строит кривые обучения для дерева заданной глубины"""
    train_sizes = np.linspace(0.1, 1.0, 10)
    train_errors = []
    val_errors = []
    
    for size in train_sizes:
        n_samples_subset = int(size * len(X_train_c))
        
        # Берем подвыборку
        X_subset = X_train_c[:n_samples_subset]
        y_subset = y_train_c[:n_samples_subset]
        
        # Обучаем модель
        tree = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
        tree.fit(X_subset, y_subset)
        
        # Ошибки
        train_error = 1 - accuracy_score(y_subset, tree.predict(X_subset))
        val_error = 1 - accuracy_score(y_test_c, tree.predict(X_test_c))
        
        train_errors.append(train_error)
        val_errors.append(val_error)
    
    return train_sizes * len(X_train_c), train_errors, val_errors

# Строим кривые для разных глубин
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, depth in enumerate([2, 5, None]):
    sizes, train_err, val_err = plot_learning_curves(depth)
    
    axes[idx].plot(sizes, train_err, 'b-o', lw=2, label='Train Error')
    axes[idx].plot(sizes, val_err, 'r-s', lw=2, label='Validation Error')
    
    depth_str = str(depth) if depth is not None else '∞'
    axes[idx].set_title(f'Глубина = {depth_str}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Размер тренировочной выборки')
    axes[idx].set_ylabel('Ошибка')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim([0, 0.5])
    
    # Добавляем комментарии
    if depth == 2:
        axes[idx].text(0.5, 0.95, 'Высокий bias\n(обе ошибки высокие)', 
                      transform=axes[idx].transAxes, fontsize=9, va='top', ha='center',
                      bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))
    elif depth is None:
        axes[idx].text(0.5, 0.95, 'Высокий variance\n(большой разрыв)', 
                      transform=axes[idx].transAxes, fontsize=9, va='top', ha='center',
                      bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))

plt.tight_layout()
plt.show()

## 6. Визуализация структуры дерева

In [ ]:
# Обучаем небольшое дерево для визуализации
tree_visual = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_visual.fit(X_train_c, y_train_c)

# Визуализация структуры
plt.figure(figsize=(16, 8))
plot_tree(tree_visual, 
          feature_names=['Биомаркер 1', 'Биомаркер 2'],
          class_names=['Здоров', 'Болен'],
          filled=True, 
          rounded=True,
          fontsize=10)
plt.title('Структура решающего дерева (max_depth=3)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nИнтерпретация узлов:")
print("  - Верхняя строка: условие разбиения (feature <= threshold)")
print("  - gini: мера неопределенности (0 = чистый узел, 0.5 = максимальная неопределенность)")
print("  - samples: количество объектов в узле")
print("  - value: [количество класса 0, количество класса 1]")
print("  - class: предсказанный класс (большинство в узле)")

## 7. Практические рекомендации

### Преимущества решающих деревьев:
✓ Интерпретируемость (можно визуализировать и объяснить)
✓ Не требуют нормализации данных
✓ Работают с категориальными и числовыми признаками
✓ Автоматический отбор признаков
✓ Нелинейные зависимости

### Недостатки:
✗ Склонность к переобучению (высокий variance)
✗ Неустойчивость (малые изменения в данных → большие изменения в дереве)
✗ Сложность обобщения на новые данные

### Решение проблем:
1. **Ограничение сложности**: max_depth, min_samples_split, min_samples_leaf
2. **Pruning (обрезка)**: ccp_alpha (cost complexity pruning)
3. **Ансамбли**: Random Forest, Gradient Boosting (следующие ноутбуки!)

### Как выбрать гиперпараметры:
- Используйте **кросс-валидацию** для оценки обобщающей способности
- **max_depth**: обычно 3-10 для интерпретируемости, до 20+ для точности
- **min_samples_split**: 2-20 (больше → меньше overfitting)
- **min_samples_leaf**: 1-10 (больше → более гладкие границы)

## 8. Сравнение с кросс-валидацией

In [ ]:
# Оценка для разных глубин с помощью кросс-валидации
depths_cv = range(1, 21)
cv_scores_mean = []
cv_scores_std = []

for depth in depths_cv:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
    scores = cross_val_score(tree, X_train_c, y_train_c, cv=5, scoring='accuracy')
    cv_scores_mean.append(scores.mean())
    cv_scores_std.append(scores.std())

# График
plt.figure(figsize=(10, 6))
plt.plot(depths_cv, cv_scores_mean, 'b-o', lw=2, label='Средняя точность (CV)')
plt.fill_between(depths_cv, 
                 np.array(cv_scores_mean) - np.array(cv_scores_std),
                 np.array(cv_scores_mean) + np.array(cv_scores_std),
                 alpha=0.2, color='b', label='±1 std')

optimal_depth_cv = list(depths_cv)[np.argmax(cv_scores_mean)]
plt.axvline(optimal_depth_cv, color='red', linestyle='--', lw=2, 
           label=f'Оптимальная глубина = {optimal_depth_cv}')

plt.xlabel('Максимальная глубина дерева', fontsize=12)
plt.ylabel('Accuracy (5-fold CV)', fontsize=12)
plt.title('Выбор оптимальной глубины с помощью кросс-валидации', fontsize=13, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n✓ Оптимальная глубина (CV): {optimal_depth_cv}")
print(f"  Точность: {max(cv_scores_mean):.2%} ± {cv_scores_std[optimal_depth_cv-1]:.2%}")

## 9. Резюме

### Bias-Variance Tradeoff для решающих деревьев:

| Модель | Bias | Variance | Применение |
|--------|------|----------|------------|
| **Мелкое дерево** (depth=1-3) | Высокий ↑ | Низкий ↓ | Быстрая интерпретация, простые правила |
| **Средняя глубина** (depth=5-8) | Средний → | Средний → | **Оптимальный баланс** для большинства задач |
| **Глубокое дерево** (depth>15 или ∞) | Низкий ↓ | Высокий ↑ | Переобучение, нужны ансамбли |

### Ключевые выводы:

1. **Решающие деревья имеют высокий variance** → склонность к переобучению
2. **Контроль сложности** через max_depth, min_samples_* критичен для обобщения
3. **Одиночные деревья нестабильны** → малые изменения в данных дают разные деревья
4. **Решение: ансамбли деревьев** (Random Forest, Gradient Boosting) → следующие ноутбуки!

### Применение в биологии:

- Решающие деревья отлично подходят для **диагностических правил**
- Можно получить **интерпретируемые биомедицинские правила**
- Для **высокой точности** лучше использовать ансамбли (Random Forest, Boosting)

## 10. Задания для самостоятельной работы

1. **Влияние других гиперпараметров**: Исследуйте влияние `min_samples_split`, `min_samples_leaf` и `max_features` на bias-variance tradeoff

2. **Pruning**: Примените cost complexity pruning (`ccp_alpha`) и сравните с ограничением глубины

3. **Реальные данные**: Используйте датасет по экспрессии генов (например, из sklearn.datasets) и постройте дерево для классификации типов рака

4. **Feature importance**: Постройте график важности признаков (`feature_importances_`) для интерпретации модели

5. **Сравнение критериев**: Сравните работу деревьев с критериями 'gini' и 'entropy' для классификации